In [138]:
import numpy as np
from _plotly_future_ import v4_subplots
import plotly.io as pio
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.io.orca.config.executable = '/anaconda2/envs/pytorch/bin/orca'
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import colorlover as cl
import pandas as pd
import sklearn.preprocessing

glob_layout = go.Layout(
    font=dict(family='Helvetica', size=18, color='black'),
    margin=dict(l=60, r=10, t=10, b=60),
    xaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True, mirror=True),
    yaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True, mirror=True),
    legend_orientation="v",
)

In [139]:
def moving_average(a, n=100):
    if len(a.shape) == 1:
        ret = np.cumsum(a, dtype=float)
        ret[n:] = ret[n:] - ret[:-n]
        return ret[n - 1:] / n
    elif len(a.shape) == 2:
        new = []
        for ii in range(a.shape[1]):
            ret = np.cumsum(a[:, ii], dtype=float)
            ret[n:] = ret[n:] - ret[:-n]
            new.append(ret[n - 1:] / n)
        return np.transpose(np.array(new))

In [143]:
import pickle
loss_dict = pickle.load(open("loss.pkl", "rb"))

g_loss = np.array(loss_dict['g_loss'])
print(g_loss.shape)

d_loss = np.array(loss_dict['d_loss'])
print(d_loss.shape)

(18530,)
(18530, 4)


In [144]:
g_loss_avrg = moving_average(g_loss)
d_loss_avrg = moving_average(d_loss)
print(g_loss_avrg.shape, d_loss_avrg.shape)

(18431,) (18431, 4)


In [145]:
trace0 = go.Scatter(
    x = list(range(g_loss_avrg.shape[0])),
    y = g_loss_avrg,
    mode = 'markers+lines',
    opacity = 0.6,
    name = "generator",
    marker = dict(
        size = 5,
        color = 'blue',
    )
)

trace1 = go.Scatter(
    x = list(range(d_loss_avrg.shape[0])),
    y = d_loss_avrg[:, 2],
    mode = 'markers+lines',
    opacity = 0.6,
    name = "discrminator",
    marker = dict(
        size = 5,
        color = 'red',
    )
)


data = [trace0, trace1]

layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': r'iteration'})
layout["yaxis"].update({'title': 'loss'})


fig = dict(data=data, layout=layout)
iplot(fig)